# PyComBat-Seq Batch Correction

 PyComBat-Seq is a method for removing batch effects from genomic data
 while preserving biological variation. It's particularly useful for
 combining data from multiple studies or sequencing batches.

## Key benefits of batch correction:
 1. Reduces technical variation between batches
 2. Improves data integration across studies
 3. Enhances downstream analysis (clustering, differential expression, etc.)
 4. Preserves biological signals of interest

## In this notebook, we'll:
 - Load data from multiple batches
 - Apply PyComBat-Seq batch correction
 - Compare data before and after batch correction
 - Visualize the results using PCA

# Set random seed for reproducibility
np.random.seed(42)

In [59]:
import pandas as pd
import anndata
import matplotlib.pyplot as plt
import numpy as np
import omicverse as ov
ov.ov_plot_set()

All dependencies are satisfied.


In [ ]:
import os
import glob
import pandas as pd

# Define the input (raw) and output (processed) folders
raw_folder = "data/batch_correction/raw2/"
processed_folder = "data/batch_correction/processed2/"

# Create the processed folder if it doesn't exist
os.makedirs(processed_folder, exist_ok=True)

# Find all files matching the pattern in the raw folder
file_pattern = os.path.join(raw_folder, "*.tsv")
files = glob.glob(file_pattern)
print("Found files:", files)

# Loop through each file
for file in files:
    # Read the file using tab as the delimiter and GeneID as index
    df = pd.read_csv(file, sep="\t")
    
    # Remove rows that contain any zero values
    df_nonzero = df[(df != 0).all(axis=1)]
    
    # Create a new filename by replacing the suffix with '_nonzero.tsv'
    base_name = os.path.basename(file)
    new_filename = base_name.replace(".tsv", "_nonzero.tsv")
    output_path = os.path.join(processed_folder, new_filename)
    
    # Save the processed DataFrame to a TSV file
    df_nonzero.to_csv(output_path, sep="\t")
    print("Processed and saved:", output_path)


In [92]:
# -------------------------------
# 1. Set up folder paths and file pattern
# -------------------------------
raw_folder = "data/batch_correction/raw/"
file_pattern = os.path.join(raw_folder, "*_raw_counts_GRCh38.p13_NCBI.tsv.gz")
files = glob.glob(file_pattern)
print("Found files:", files)

# Define a mapping from filename prefix to batch label.
# Adjust the mapping based on your file naming convention.
mapping = {
    "GSE162760": "1",
    "GSE82177": "2",
    "GSE83083": "3"
}

# -------------------------------
# 2. Load each file, process it, and create an AnnData object
# -------------------------------
adatas = []
for file in files:
    # Extract sample name from the filename
    sample = os.path.basename(file).replace("_raw_counts_GRCh38.p13_NCBI.tsv.gz", "")
    
    # Read the dataset
    df = pd.read_csv(file, sep="\t", index_col=0)
    
    # Optional: If the dataset has multiple columns, select only the first count column.
    if df.shape[1] > 1:
        df = df.iloc[:, 0:1]
    
    # Apply log1p transformation
    df_log = np.log1p(df)
    
    # Create an AnnData object from the log-transformed data (transpose so genes become columns)
    adata = anndata.AnnData(df_log.T)
    
    # Assign the batch label using the mapping dictionary; default to "unknown" if not found.
    adata.obs["batch"] = mapping.get(sample, "unknown")
    
    adatas.append(adata)
    print(f"Processed sample {sample} with shape {adata.shape}")

# -------------------------------
# 3. Concatenate the AnnData objects into one
# -------------------------------
adata_merged = anndata.concat(adatas, merge="same")
print("Merged AnnData object:")
print(adata_merged)

# -------------------------------
# 4. Convert the merged AnnData object to a DataFrame and remove rows with any zero values
# -------------------------------
# We assume that adata_merged.X is dense.
merged_df = pd.DataFrame(adata_merged.X, index=adata_merged.obs_names, columns=adata_merged.var_names)
# Remove rows (genes) that contain any zero values
merged_df = merged_df[(merged_df != 0).all(axis=1)]
print("Merged DataFrame shape after removing zero rows:", merged_df.shape)

# -------------------------------
# 5. Save the final merged DataFrame to a TSV file
# -------------------------------
output_path = "data/merged_counts_with_annotation.tsv"
merged_df.to_csv(output_path, sep="\t")
print(f"Merged counts saved to '{output_path}'")

AnnData object with n_obs × n_vars = 138 × 12915
    obs: 'batch'
Merged counts saved to 'data/merged_counts_with_annotation.tsv'


In [94]:
print(merged_df.head())

GeneID        653635  100996442    729737  102723897  100132287  113219467  \
GSM4959389  5.686975   5.141664  4.882802   5.866468   4.094345   6.813445   
GSM4959390  5.765191   5.805135  4.990433   5.846439   3.951244   6.049733   
GSM4959391  6.291569   5.849325  5.043425   6.366470   4.543295   7.213032   
GSM4959392  4.859812   5.521461  4.882802   4.941642   4.110874   6.350886   
GSM4959393  5.099866   5.438079  4.060443   5.318120   3.044522   6.369901   

GeneID      100133331  100288069  105378580    643837  ...      4538  \
GSM4959389   3.850148   4.859812   4.948760  6.352629  ...  8.382061   
GSM4959390   4.077537   5.513429   5.220356  6.107023  ...  7.944492   
GSM4959391   4.488636   5.480639   5.505332  6.480045  ...  8.821142   
GSM4959392   4.110874   5.105945   5.153292  5.823046  ...  7.771910   
GSM4959393   2.890372   4.852030   5.181784  5.375278  ...  7.518607   

GeneID          4564      4575      4568      4540      4541      4556  \
GSM4959389  4.976734  5.

In [113]:
raw_data=adata.to_df().T
raw_data.head()

,GSM4959389,GSM4959390,GSM4959391,GSM4959392,GSM4959393,GSM4959394,GSM4959395,GSM4959396,GSM4959397,GSM4959398,...,GSM2192697,GSM2192698,GSM2192699,GSM2192700,GSM2192701,GSM2192702,GSM2192703,GSM2192704,GSM2192705,GSM2192706
GeneID,,,,,,,,,,,,,,,,,,,,,
653635,5.686975,5.765191,6.291569,4.859812,5.099866,6.315358,5.730100,5.609472,5.680173,6.340359,...,5.407172,5.342334,5.420535,5.220356,5.587249,5.384495,5.267858,5.220356,5.164786,5.659482
100996442,5.141664,5.805135,5.849325,5.521461,5.438079,6.086775,5.723585,5.996452,5.549076,5.937536,...,3.761200,3.465736,3.401197,3.295837,3.828641,3.526361,3.332205,3.401197,3.496508,3.555348
729737,4.882802,4.990433,5.043425,4.882802,4.060443,5.533389,5.505332,5.583496,5.087596,5.517453,...,5.420535,5.389072,5.568345,5.342334,5.451038,5.446737,5.111988,5.204007,5.192957,5.529429
102723897,5.866468,5.846439,6.366470,4.941642,5.318120,6.448889,5.814131,5.780744,5.733341,6.472346,...,5.476464,5.468060,5.501258,5.361292,5.669881,5.517453,5.384495,5.247024,5.393628,5.749393
100132287,4.094345,3.951244,4.543295,4.110874,3.044522,4.867534,4.804021,4.859812,4.521789,4.836282,...,4.564348,4.553877,4.510860,4.369448,4.624973,4.454347,4.189655,4.317488,4.317488,4.663439


In [114]:
removing_data=adata.to_df(layer='batch_correction').T
removing_data.head()

,GSM4959389,GSM4959390,GSM4959391,GSM4959392,GSM4959393,GSM4959394,GSM4959395,GSM4959396,GSM4959397,GSM4959398,...,GSM2192697,GSM2192698,GSM2192699,GSM2192700,GSM2192701,GSM2192702,GSM2192703,GSM2192704,GSM2192705,GSM2192706
GeneID,,,,,,,,,,,,,,,,,,,,,
653635,5.047280,5.116929,5.585656,4.310711,4.524473,5.606839,5.085681,4.978265,5.041222,5.629102,...,5.647873,5.538497,5.670416,5.332728,5.951650,5.609619,5.412861,5.332728,5.238985,6.073503
100996442,4.021340,4.350658,4.372592,4.209855,4.168468,4.490452,4.310181,4.445620,4.223562,4.416377,...,4.467410,4.336469,4.307868,4.261175,4.497297,4.363336,4.277292,4.307868,4.350106,4.376182
729737,4.635222,4.708977,4.745290,4.635222,4.071695,5.081042,5.061815,5.115378,4.775559,5.070121,...,5.027268,4.991485,5.195371,4.938331,5.061959,5.057068,4.676360,4.781012,4.768445,5.151112
102723897,5.220671,5.202582,5.672238,4.385434,4.725442,5.746673,5.173404,5.143251,5.100441,5.767858,...,5.656723,5.642601,5.698393,5.463169,5.981776,5.725609,5.502163,5.271133,5.517511,6.115402
100132287,3.853996,3.755724,4.162305,3.865347,3.133049,4.384971,4.341354,4.379668,4.147536,4.363509,...,4.422653,4.407860,4.347091,4.147322,4.508295,4.267257,3.893333,4.073920,4.073920,4.562636


In [115]:
raw_data.to_csv('raw_data.csv')
removing_data.to_csv('removing_data.csv')